<a href="https://colab.research.google.com/github/anumadhyani/mywork/blob/master/Detect_image_real_or_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests

In [2]:
!pip install pillow

In [29]:
import os
from PIL import Image
import requests
from io import BytesIO
import hashlib # Added for generating unique filenames

from google.colab import userdata
unsplashkey = userdata.get('Unsplash_Key')

from google.colab import drive
drive.mount('/content/drive', force_remount= True)

# Define the path where you want to save your images in Google Drive
GDRIVE_SAVE_PATH = '/content/drive/MyDrive/Unsplash_Images'
print(f"Images will be saved to: {GDRIVE_SAVE_PATH}")

# Ensure the directory exists
os.makedirs(GDRIVE_SAVE_PATH, exist_ok=True)

print("a")

def fetch_image_urls(query):

  api_url =  f"https://api.unsplash.com/search/photos/?query={query}&client_id={unsplashkey}"
  responses = requests.get(api_url)

  if responses.status_code != 200:
    print("Error fetching image data")
    return

  data = responses.json()
  print("hi")
  urls = [result["urls"]["regular"] for result in data["results"]]
  print("here")
  return urls

def download_and_save_images(image_url, base_save_path): # Renamed parameter for clarity
  print(f"Attempting to download: {image_url}") # Clarified print statement
  response = requests.get(image_url)
  if response.status_code == 200:
    image = Image.open(BytesIO(response.content))
    #Determine file extension from content type or default to .jpg
    content_type = response.headers.get('content-type', 'image/jpeg')
    extension = '.' + content_type.split('/')[-1]

    # Handle cases where content type might not provide a standard extension, or add more common types
    if extension not in ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp']:
      extension = '.jpg' # Fallback extension

    # Generate a unique filename using a hash of the URL
    filename = hashlib.md5(image_url.encode('utf-8')).hexdigest() + extension
    full_save_path = os.path.join(base_save_path, filename)

    print(f"Saving image to: {full_save_path}")
    image.save(full_save_path)
    print(f"Successfully saved to {full_save_path}")
  else:
    print(f"Failed to download image from {image_url}. Status code: {response.status_code}")

urls = fetch_image_urls("coding on laptop")
if urls:
  print(f"Found {len(urls)} image URLs.")
  for url in urls:
    download_and_save_images(url, GDRIVE_SAVE_PATH)


Mounted at /content/drive
Images will be saved to: /content/drive/MyDrive/Unsplash_Images
a
hi
here
Found 10 image URLs.
Attempting to download: https://images.unsplash.com/photo-1659642136446-41ab12578991?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w4NTgxMjh8MHwxfHNlYXJjaHwxfHxjb2RpbmclMjBvbiUyMGxhcHRvcHxlbnwwfHx8fDE3Njg1NTM1NDF8MA&ixlib=rb-4.1.0&q=80&w=1080
Saving image to: /content/drive/MyDrive/Unsplash_Images/6f23b50b9a91d8e3cc3b273eb3d7be31.jpeg
Successfully saved to /content/drive/MyDrive/Unsplash_Images/6f23b50b9a91d8e3cc3b273eb3d7be31.jpeg
Attempting to download: https://images.unsplash.com/photo-1569693799105-4eb645d89aea?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w4NTgxMjh8MHwxfHNlYXJjaHwyfHxjb2RpbmclMjBvbiUyMGxhcHRvcHxlbnwwfHx8fDE3Njg1NTM1NDF8MA&ixlib=rb-4.1.0&q=80&w=1080
Saving image to: /content/drive/MyDrive/Unsplash_Images/7d30edb29dbfbc8dec8475b87338a5c2.jpeg
Successfully saved to /content/drive/MyDrive/Unsplash_Images/7d30edb29dbfbc8dec8475b87338a5c2.jpeg
Attem

First, we need to mount your Google Drive to be able to save files there. You will be prompted to authorize this connection.

Mounted at /content/drive
Images will be saved to: /content/drive/MyDrive/Unsplash_Images
